In [7]:
import sys
sys.path.append('../topic_model/')

In [8]:
import cProfile
import pstats
from pstats import SortKey
import sys
import pandas as pd


import json, os

with open('../topic_model/data/doc_vecs.json') as f:
    doc_strs =  f.read().split('\n')
    docs = []
    for idx, doc_str in enumerate(doc_strs):
#         if idx == 200:
#             break 
        if doc_str:
            doc = json.loads(doc_str)
            docs.append(doc['paragraphs'])

vocab_fp = os.path.join('../topic_model/data/', 'vocab.txt')
vocab = open(vocab_fp).read().split('\n')

In [9]:
import sampler_cy
from importlib import reload

In [10]:
s2 = sampler_cy.BOW_Paragraph_GibbsSampler(docs=docs[:20000], vocab=vocab)
s2.initialize()

In [4]:
s2.sample_pass()
s2.pythonize_vars()

In [7]:
s2.par_doc_to_type

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
s2.switching_variable_counts.shape

(8856, 2)

In [ ]:
cProfile.run('s2.sample_pass()', 'restats')
p = pstats.Stats('restats')
p.strip_dirs().sort_stats(2).print_stats()

# Cython testing

In [5]:
%load_ext Cython

In [6]:
%%cython
from libc.stdlib cimport rand, RAND_MAX
import numpy as np 

cdef double random_double(double max_num) nogil:
    return (rand() / (RAND_MAX + 1) * max_num)

cdef int categorical(int k, double[:] p) nogil:
    cdef double sum_of_weight = 0;
    cdef int i
    for i in range(k):
        sum_of_weight += p[i];
    cdef double rnd = random_double(sum_of_weight);
    for i in range(k):
        if rnd < p[i]:
            return i;
        rnd -= p[i];
        
def init():
    cdef double[:] partype_prob_vec = np.ones(10, dtype=np.float64)
    cdef int partype = categorical(10, p=partype_prob_vec)
    return partype

In [7]:
t = []
for i in range(100000):
    t.append(init())

In [8]:
pd.Series(t).value_counts()

6    10207
8    10179
4    10106
1    10074
9    10029
3     9990
0     9950
5     9944
7     9762
2     9759
dtype: int64

In [1]:
%load_ext Cython

In [10]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

In [23]:
%%cython 

from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

cdef class SomeMemory:
    cdef double* data
    def __cinit__(self, size_t number):
        # allocate some memory (uninitialised, may contain arbitrary data)
        self.data = <double*> PyMem_Malloc(number * sizeof(double))
        if not self.data:
            raise MemoryError()

    def resize(self, size_t new_number):
        # Allocates new_number * sizeof(double) bytes,
        # preserving the current content and making a best-effort to
        # re-use the original data location.
        mem = <double*> PyMem_Realloc(self.data, new_number * sizeof(double))
        if not mem:
            raise MemoryError()
        # Only overwrite the pointer if the memory was really reallocated.
        # On error (mem is NULL), the originally memory has not been freed.
        self.data = mem

    def get_data(self):
        return self.data[0]

    def __dealloc__(self):
        PyMem_Free(self.data)  # no-op if self.data is NULL

In [30]:
a = SomeMemory(5)

In [31]:
a.get_data()

1.3591251795914e-311

In [10]:
%%cython

import numpy as np 

DTYPE = np.intc

cdef int clip(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)

def compute(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    # array_1.shape is now a C array, no it's not possible
    # to compare it simply by using == without a for-loop.
    # To be able to compare it to array_2.shape easily,
    # we convert them both to Python tuples.
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

In [11]:
%%cython 
import numpy as np 

DTYPE = np.intc

cdef class SomeMemoryNp:
    cdef int[:] data
    cdef list a, b
    cdef int[:, :] c, d
    
    def __cinit__(self, double number):
        self.data = np.zeros(int(number), dtype=DTYPE)
        
    def get_data(self):
        return self.data[0]

In [62]:
%%cython 

import numpy as np 
cimport numpy as np 

cdef int categorical(int k, np.ndarray[np.float64_t, ndim=1] p):
    return np.random.choice(range(k), p=p)

cdef int bernoilli(float p):
    return np.random.binomial(1, p)

def test_cat():
    return categorical(5, np.array([.2, .2, .2, .2, .2]))

In [130]:
import numpy as np 
np.zeros

<function numpy.zeros>

In [109]:
%%cython

cdef int t() nogil:
    cdef int[2] p
    p[0] = 0
    p[1] = 1
    return 1

In [123]:
%%cython 

cdef int[2] p
p[0] = 1 

In [143]:
%%cython

from libc.stdlib cimport rand, RAND_MAX

cdef int random_int(int max_num) nogil:
    return <int>(rand() / (RAND_MAX + 1) * max_num)

cdef float random_float(int max_num) nogil:
    return (rand() / (RAND_MAX + 1) * max_num)

cdef int categorical(int k, int[:] p) nogil:
    cdef int sum_of_weight = 0;
    cdef int i
    for i in range(k):
        sum_of_weight += p[i];
    cdef int rnd = random_int(sum_of_weight);
    for i in range(k):
        if rnd < p[i]:
            return i;
        rnd -= p[i];

cdef int bernoilli(float p) nogil:
    cdef float r = random_float(1)
    if r < p:
        return 1
    else:
        return 0
    
def b():
    return bernoilli(.7)

In [157]:
bs = []
for i in range(1000000):
    bs.append(b())

In [158]:
pd.Series(bs).value_counts()

1    699440
0    300560
dtype: int64

In [6]:
%%cython

from libc.math cimport log, exp 
def logger():
    return exp(log(10))

In [167]:
%%cython

from libc.stdlib cimport rand, RAND_MAX


cdef float random(float max_num) nogil:
    return (rand() / (RAND_MAX + 1) * max_num)

cdef int categorical(int k, float[:] p) nogil:
    cdef float sum_of_weight = 0;
    cdef int i
    for i in range(k):
        sum_of_weight += p[i];
    cdef float rnd = random(sum_of_weight);
    for i in range(k):
        if rnd < p[i]:
            return i;
        rnd -= p[i];

cdef int bernoilli(float p) nogil:
    cdef float r = rand() / (RAND_MAX + 1)
    if r < p:
        return 1
    else:
        return 0

In [163]:
%%cython

cimport numpy as np 
import numpy as np 

DTYPE = np.intc

cdef int l(int[:] a) nogil:
    return len(a)

def t():
    cdef int[:] a = np.zeros(4, dtype=DTYPE)
    return l(a)

In [3]:
%load_ext Cython

In [76]:
%%cython

import numpy as np
cimport numpy as np 
import os
import json
from tqdm import tqdm
import pickle
import glob, re
import random
from cython.parallel import prange
from libc.math cimport log, exp 
from libc.stdlib cimport rand, RAND_MAX
import cython 
from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free

DTYPE = np.intc

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef float random_double(double  max_num) nogil:
    return (rand() / (RAND_MAX + 1) * max_num)

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef int categorical(int k, double[:] p) nogil:
    cdef double sum_of_weight = 0;
    cdef int i
    for i in range(k):
        sum_of_weight += p[i];
    cdef double rnd = random_double(sum_of_weight);
    for i in range(k):
        if rnd < p[i]:
            return i;
        rnd -= p[i];

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
cdef int bernoilli(float p) nogil:
    cdef float r = rand() / (RAND_MAX + 1)
    if r < p:
        return 1
    else:
        return 0


cdef class BOW_Paragraph_GibbsSampler():
    cdef int ***docs 
    cdef int[:] num_pars_per_doc
    cdef int **num_words_per_par_per_doc

    cdef Py_ssize_t n_docs, n_graphs, n_vocab, num_partypes, num_topics, max_graphs_per_doc
    cdef float H_T, H_P, gamma, alpha, beta
    
    cdef int[:] partype_counts, doc__wordtopic_counts, partype__wordtopic_counts, wordtopic__word_counts
    cdef int[:, :] doc_by_wordtopic__wordtopic_counts, partype_by_wordtopic__wordtopic_counts, vocab_by_wordtopic__word_counts
    cdef int[:, :] pardoc_to_type, old_pardoc_to_type
    cdef int*** word_to_topic
    cdef int*** switching_variable
    cdef int*** old_switching_variable

    def __cinit__(
        self, list docs, list vocab, Py_ssize_t num_partypes=10, Py_ssize_t num_topics=25, 
        float H_P_prior=1, float H_T_prior=1, float gamma=.7, float alpha=1., float beta=1.
    ):
        # copy word-level and paragraph-level data
        self.docs = <int***>PyMem_Malloc(len(docs) * sizeof(int**))
        self.word_to_topic = <int***>PyMem_Malloc(len(docs) * sizeof(int**))
        self.switching_variable = <int***>PyMem_Malloc(len(docs) * sizeof(int**))
        self.old_switching_variable = <int***>PyMem_Malloc(len(docs) * sizeof(int**))
        self.num_words_per_par_per_doc = <int**>PyMem_Malloc(len(docs) * sizeof(int**))
        cdef int i, j, k
        for i in range(len(docs)):
            self.docs[i] = <int**> PyMem_Malloc(len(docs[i]) * sizeof(int*))
            self.word_to_topic[i] = <int**> PyMem_Malloc(len(docs[i]) * sizeof(int*))
            self.switching_variable[i] = <int**> PyMem_Malloc(len(docs[i]) * sizeof(int*))
            self.old_switching_variable[i] = <int**> PyMem_Malloc(len(docs[i]) * sizeof(int*))
            self.num_words_per_par_per_doc[i] = <int*> PyMem_Malloc(len(docs[i]) * sizeof(int))
            for j in range(len(docs[i])):
                self.docs[i][j] = <int*> PyMem_Malloc(len(docs[i][j]) * sizeof(int))
                self.word_to_topic[i][j] = <int*> PyMem_Malloc(len(docs[i][j]) * sizeof(int))
                self.switching_variable[i][j] = <int*> PyMem_Malloc(len(docs[i][j]) * sizeof(int))
                self.old_switching_variable[i][j] = <int*> PyMem_Malloc(len(docs[i][j]) * sizeof(int))
                self.num_words_per_par_per_doc[i][j] = len(docs[i][j])
                for k in range(len(docs[i][j])):
                    self.docs[i][j][k] = docs[i][j][k]
            
        self.num_pars_per_doc = np.array(list(map(lambda x: len(x), docs)), dtype=DTYPE)
        self.n_docs = len(docs)
        self.n_graphs = sum(list(map(lambda x: len(x), docs)))
        self.max_graphs_per_doc = max(list(map(lambda x: len(x), docs)))
        self.n_vocab = len(vocab)

        # hyperparameters
        self.num_partypes = num_partypes
        self.num_topics = num_topics

        # priors
        self.H_T = H_T_prior
        self.H_P = H_P_prior
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        
        # counts 
        ## doc-type probability
        #### I(doc d has type t)
        #### count_t_d
        self.partype_counts = np.zeros(self.num_partypes, dtype=DTYPE)
        self.pardoc_to_type = np.zeros((self.n_docs, self.max_graphs_per_doc), dtype=DTYPE)
        self.old_pardoc_to_type = np.zeros((self.n_docs, self.max_graphs_per_doc), dtype=DTYPE)

        ## background word-topic probabilities
        #### I(doc d has type t, word w has topic k)
        #### count_k_d_t_w
        self.doc__wordtopic_counts = np.zeros(self.n_docs, dtype=DTYPE)
        self.doc_by_wordtopic__wordtopic_counts = np.zeros((self.n_docs, self.num_topics), dtype=DTYPE)

        ## paragraph word-topic probabilities
        #### I(source S_d in doc d has type s, word w has topic k)
        #### count_k_d_s_Sd_w
        self.partype__wordtopic_counts = np.zeros(self.num_partypes, dtype=DTYPE)
        self.partype_by_wordtopic__wordtopic_counts = np.zeros((self.num_partypes, self.num_topics), dtype=DTYPE)

        ## word probabilities
        self.vocab_by_wordtopic__word_counts = np.zeros((self.n_vocab, self.num_topics), dtype=DTYPE)
        self.wordtopic__word_counts = np.zeros(self.num_topics, dtype=DTYPE)

    def initialize(self):
        ## probability vectors
        cdef double[:] partype_prob_vec = np.ones(self.num_partypes, dtype=np.float64)
        cdef double[:] z_prob_vec = np.ones(self.num_topics, dtype=np.float64)

        cdef doc_id, par_id, word
        cdef int** doc
        cdef int* par
        ## iterate through documents
        for doc_id in range(self.n_docs):
            doc = self.docs[doc_id]
            for par_id in range(self.num_pars_per_doc[doc_id]):
                par = doc[par_id]
                partype = categorical(self.num_partypes, p=partype_prob_vec)
                self.partype_counts[partype] += 1
                self.pardoc_to_type[doc_id, par_id] = partype
                self.old_pardoc_to_type[doc_id, par_id] = partype

                for word_id in range(self.num_words_per_par_per_doc[doc_id][par_id]):
                    word = par[word_id]
                    s = bernoilli(self.gamma)
                    self.switching_variable[doc_id][par_id][word_id] = s
                    self.old_switching_variable[doc_id][par_id][word_id] = s

                    word_topic = categorical(self.num_topics, p=z_prob_vec)
                    if s == 0:
                        ## set paragraph word-topics counts
                        self.partype_by_wordtopic__wordtopic_counts[partype, word_topic] += 1
                        self.partype__wordtopic_counts[partype] += 1
                        self.vocab_by_wordtopic__word_counts[word, word_topic] += 1
                        self.wordtopic__word_counts[word_topic] += 1

                    else:
                        ## set document word-topic counts
                        self.doc_by_wordtopic__wordtopic_counts[doc_id, word_topic] += 1
                        self.doc__wordtopic_counts[doc_id] += 1
                        self.vocab_by_wordtopic__word_counts[word, word_topic] += 1
                        self.wordtopic__word_counts[word_topic] += 1

                    ## cache
                    self.word_to_topic[doc_id][par_id][word_id] = word_topic


    def __dealloc__(self):
        PyMem_Free(self.docs)  # no-op if self.data is NULL
        PyMem_Free(self.num_words_per_par_per_doc)
        PyMem_Free(self.switching_variable)
        PyMem_Free(self.old_switching_variable)
        PyMem_Free(self.word_to_topic)

In [77]:
s2 = BOW_Paragraph_GibbsSampler(docs=docs[:100], vocab=vocab)

In [78]:
s2.initialize()

In [33]:
s2.test()

In [1]:
%load_ext Cython

In [5]:
%%cython
def test(arr):
    cdef double[:] output_arr = arr
    return output_arr

cdef void test2():
    a=1

In [13]:
import numpy as np
test(np.array([1,2,3], dtype=np.float))

<MemoryView of 'ndarray' at 0x1bde237ba38>

In [3]:
%%cython

from cpython.mem cimport PyMem_Malloc, PyMem_Realloc, PyMem_Free
def test():
    
    cdef int **array = <int**>PyMem_Malloc(10 * sizeof(int*))
    array[0] = <int*> PyMem_Malloc(10*sizeof(int))
    array[1] = <int*> PyMem_Malloc(20*sizeof(int))
    
    array[0][0] = 5
    array[1][1000] = 10
    return (array[0][0], array[1][10])
#     PyMem_Free(a)

In [30]:
%%cython
cimport numpy as np
import numpy as np
cdef t():
    cdef double[:] a = np.zeros(10)
    return a
    
def t2():
    return np.asarray(t())